In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from datetime import datetime
from pandas import to_numeric, to_datetime, to_timedelta, DataFrame

In [2]:
# - Create this w/ user input for their user_name, chess_data_file (.pgn as format), & time back to go to predict - just stats of their data
# - Create a program that asks user for their user_name, chess_data_file, time back to go to predict, move by move gives them a % chance of winning.

# ### Current Considered Variables:

# > - y = 'result' <- Prediction
# - 'day': Done
# - 'time_control': Done
# - 'color': Done
# - 'pre_elo': Done
# - 'pre_diff': Done
# - 'opp_pre_elo': Done
# - 'num_moves': Done
# - 'avg_move_time' <- Create column from subtracting last time and total time devide by # of moves
# - 'opp_avg_move_time' <- Create column from subtracting last time and total time devide by # of moves
# - 'avg_80_move_time' <- Create column from subtracting last time and total time devide by # of moves (trim top 10% & bottom 10%)
# - 'opp_avg_80_move_time' <- Create column from subtracting last time and total time devide by # of moves (trim top 10% & bottom 10%)
# - 'max_move_time' <- Create column from max of all move times
# - 'opp_max_move_time' <- Create column from max of all move times
# - 'castled_on_move' <- Create column that looks for '0-0' or '0-0-0'
# - 'Opp_castled_on_move' <- Create column that looks for '0-0' or '0-0-0'
# - 'Castled_at' <- Create column that looks for '0-0' or '0-0-0'. 1 if '0-0', 0 if '0-0-0', -1 if did not castle
# - 'weighted_score'
#     - 'time': result-0.1 for win (Final_score<=-5), result+0.1 for loss (Final_score>=5)
#     - 'abandoned': result-0.2 for win (Final_score<=-5), result+0.2 for loss (Final_score>=5)
#     - 'stalemate': result-0.25 (Final_score<=-5), result+0.25 (Final_score>=5)
# - 'score_5' - Score after first 5 moves
# - 'score_10' - Score after first 10 moves
# - 'score_15' - Score after first 15 moves
# - 'score_20' - Score after first 20 moves
# - 'score_25' - Score after first 25 moves
# - 'score_30' - Score after first 30 moves
# - 'score_35' - Score after first 35 moves
# - 'score_40' - Score after first 40 moves
# - 'score_45' - Score after first 45 moves
# - 'score_50' - Score after first 50 moves
# - 'score_55' - Score after first 55 moves
# - 'score_60' - Score after first 60 moves

# ### Future Considered Variables:
# > - y = 'win%' <- Prediction
# - 'timed_game': user_input -- *'enter time in minutes per player (10 for 10 min, 5 for 5 min, etc.): '*
# - 'color': user_input -- *'enter your peices color: '*
# - 'pre_diff': user_input
# - 'Current_move_count': Done
# - 'delta_avg_move_time' <- Create column from subtracting last time and total time devide by # of moves
# - 'delta_max_move_time' <- Create column from max of all move times
# - 'delta_castled_on_move' <- Create column that looks for '0-0' or '0-0-0'
# - 'score' - has to keep updating after every move

In [8]:
def initial_chess_data(filename):
    # change filename to chess_data_file
    with open(filename, 'r+') as file:
        icd_l = file.readlines()
    icd_t = " ".join(num for num in icd_l if len(num) > 4)
    return icd_t

icd_text = initial_chess_data('dest.pgn')
#icd_text

In [4]:
def chess_data_cleanup(chess_text):
    chess_text = chess_text.replace('[', "").replace(']', "")
    chess_text = chess_text.replace('\n', ' ')
    chess_text = chess_text.replace('   ', ' ').replace('  ', ' ')
    chess_text = chess_text.replace('... ', 'b":["').replace('. ', 'w":["')
    chess_text = chess_text.replace('", ', '", "').replace(' {%clk ', '", "')
    chess_text = chess_text.replace(' {%clk', '", "')
    chess_text = chess_text.replace('}', '"],').replace('"], ', '"], "')
    chess_text = chess_text.replace(' Site "Chess.com" D', ', D')
    chess_text = chess_text.replace('Event ', '}~{"Event":')
    chess_text = chess_text.replace('", Date ', '", "Date": ')
    chess_text = chess_text.replace('" Result ', '", "Result": ')
    chess_text = chess_text.replace('" Round ', '", "Round": ')
    chess_text = chess_text.replace('" White ', '", "White": ')
    chess_text = chess_text.replace('" Black ', '", "Black": ')
    chess_text = chess_text.replace('" WhiteElo ', '", "WhiteElo": ')
    chess_text = chess_text.replace('" TimeControl ', '", "TimeControl": ')
    chess_text = chess_text.replace('" EndTime ', '", "EndTime": ')
    chess_text = chess_text.replace('" BlackElo ', '", "BlackElo": ').replace('" Termination ', '", "Termination": ')
    chess_text = chess_text.replace('"Event":"10|0 Blitz", "Date": "2017.02.16", "Round": "5", "White": "TrueMoeG", "Black": "naggvk", "Result": "0-1", "WhiteElo": "784", "BlackElo": "1210", "TimeControl": "600", "EndTime": "12:27:07 PST", "Termination": "naggvk won - game abandoned" }~{', '')
    chess_text = chess_text.replace('"Event":"Live Chess", "Date": "2017.02.20", "Round": "-", "White": "30mate", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "820", "BlackElo": "878", "TimeControl": "600", "EndTime": "11:55:45 PST", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace('"Event":"Live Chess", "Date": "2018.07.14", "Round": "-", "White": "Bran17", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "1205", "BlackElo": "961", "TimeControl": "300", "EndTime": "18:02:56 PDT", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace('"Event":"Live Chess", "Date": "2017.04.26", "Round": "-", "White": "nerbenator", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "829", "BlackElo": "842", "TimeControl": "600", "EndTime": "16:49:05 PDT", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace('"Event":"10|0 Blitz", "Date": "2017.02.16", "Round": "4", "White": "TrueMoeG", "Black": "Shuzakhan", "Result": "0-1", "WhiteElo": "845", "BlackElo": "1183", "TimeControl": "600", "EndTime": "11:58:21 PST", "Termination": "Shuzakhan won by resignation" }~{', '')
    chess_text = chess_text.replace(' PST', '').replace(' PDT', '')
    chess_text = chess_text.replace('180+2', '180').replace('300+5', '300')
    chess_text = chess_text.replace('   ', ' ').replace('  ', ' ')
    chess_text = chess_text.replace('" 1w":[', '"}~{"1w":[')
    chess_text = chess_text.replace('" 1w":"', '"}~{"1w":"')
    chess_text = chess_text.replace(', "1/2-1/2 }~{', '}~{')
    chess_text = chess_text.replace(', "1-0 }~{', '}~{')
    chess_text = chess_text.replace(', "0-1 }~{', '}~{')
    chess_text = chess_text.replace(', "1-0 ', '}').replace(', "}', '}')
    return chess_text

clean_chess_text = chess_data_cleanup(icd_text)
# clean_chess_text

In [5]:
def initial_chess_df(chess_text):
    cl = ''.join([num for num in chess_text]).split("~")
    df = DataFrame(data=cl)
    df.columns = ['a']
    abcd = [num for num in df.a if len(num) > 3]
    df = DataFrame(data=abcd)
    df.columns = ['a']
    return df

cdfi = initial_chess_df(clean_chess_text)
# cdfi

In [6]:
def initial_data_cleaning(df):
    ddf = df[df.index % 2 == 0].reset_index().drop(columns=['index'])
    mdf = df[df.index % 2 == 1].reset_index().drop(columns=['index'])

    ddf = DataFrame([literal_eval(num) for num in ddf.a])
    mdf = DataFrame([literal_eval(num) for num in mdf.a])

    ddf = ddf.rename(columns={'Date': 'date', 'Event': 'event', 'Round': 'rd',
                              'White': 'white', 'Black': 'black', 'Result': 'result',
                              'WhiteElo': 'white_elo', 'BlackElo': 'black_elo',
                              'TimeControl': 'time_control', 'EndTime': 'end_time',
                              'Termination': 'termination'}).drop(columns=['event', 'rd'])

    ddf['num_moves'] = mdf.count(axis=1)

    moves_column_names = np.array(['00' + num if len(num) == 2 
                                   else num for num in list(mdf)])
    
    moves_column_names = np.array(['0' + num if len(num) == 3 
                                   else num for num in moves_column_names])
    
    moves_column_names = np.array([num.replace('w', 'a') for 
                                   num in moves_column_names])
    
    mdf.columns = [num for num in moves_column_names]
    moves_column_names = sorted(moves_column_names)
    mdf = mdf[[num for num in moves_column_names]]

    len_mdf = len(mdf)-1
    tdf = DataFrame()
    for num in list(mdf):
        tdf[f'{num}-t'] = pd.Series(mdf[num])

    ddf['time_control'] = to_numeric(ddf['time_control'])
    ddf['date_time'] = to_datetime(ddf['date'] + ' ' + ddf['end_time'])
    ddf['date'] = to_datetime(ddf['date'])
    ddf['end_time'] = to_timedelta(ddf['end_time'])
    ddf['weekday'] = ddf.date.apply(lambda x: x.dayofweek)
    ddf['day'] = ddf.date.apply(lambda x: x.day)

    ddf['color'] = np.where(ddf['white'] == 'TrueMoeG', '1', '0')
    ddf['color'] = to_numeric(ddf['color'])
    ddf = ddf.drop(columns=['white', 'black'])
    ddf['white_elo'] = to_numeric(ddf['white_elo'])
    ddf['black_elo'] = to_numeric(ddf['black_elo'])

    ddf['post_elo'] = np.where(ddf['color'] == 1, 
                               ddf['white_elo'], ddf['black_elo'])
    
    ddf['opp_post_elo'] = np.where(ddf['color'] == 0, 
                                   ddf['white_elo'], ddf['black_elo'])
    
    ddf = ddf.drop(columns=['white_elo', 'black_elo'])
    ddf['pre_elo'] = pd.Series(
        [ddf['post_elo'][i] if i > 0 else 900 for i in ddf.index-1])
    
    ddf['elo_delta'] = ddf['post_elo'] - ddf['pre_elo']
    ddf['opp_pre_elo'] = ddf['opp_post_elo'] - ddf['elo_delta']
    ddf['diff'] = ddf['pre_elo'] - ddf['opp_pre_elo']
    ddf = ddf.drop(columns=['post_elo', 'opp_post_elo', 'elo_delta'])

    ddf['winner'] = ddf['termination'].str.extract(
        '(^[a-zA-Z0-9]+)', expand=False)
    
    ddf['won_by'] = ddf['termination'].str.extract(
        '([a-zA-Z0-9]+$)', expand=False)
    
    ddf['result'] = np.where(ddf['winner'] == 'TrueMoeG', 1.0, 
                             np.where(ddf['winner'] == 'Game', 0.5, 0.0))
    
    ddf = ddf.drop(columns=['termination', 'winner'])

#     for col_name in list(mdf):
#         for val in mdf[col_name]:
#             mdf[col_name] = pd.Series([ast.literal_eval(val) for val in mdf[col_name]])

#     num_to_use_later = len(list(mdf))


#     mdf = pd.DataFrame([ast.literal_eval(num) for num in mdf])
#     moves_column_names = list(mdf)
#     for col_name in list(tdf):
#         for val in tdf[col_name]:
#             tdf[col_name] = pd.Series([ast.literal_eval(val[1].astype()) for val in tdf[col_name]])
#     modify: ['date to day', 'end_time to start_time', 'won_by: try assigning
#     different values to all outcomes to see if a model fits it']

    return ddf, mdf, tdf

dfi, mfi, tfi = initial_data_cleaning(cdfi)
dfi.info()
dfi.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810 entries, 0 to 1809
Data columns (total 13 columns):
date            1810 non-null datetime64[ns]
end_time        1810 non-null timedelta64[ns]
result          1810 non-null float64
time_control    1810 non-null int64
num_moves       1810 non-null int64
date_time       1810 non-null datetime64[ns]
weekday         1810 non-null int64
day             1810 non-null int64
color           1810 non-null int64
pre_elo         1810 non-null int64
opp_pre_elo     1810 non-null int64
diff            1810 non-null int64
won_by          1810 non-null object
dtypes: datetime64[ns](2), float64(1), int64(8), object(1), timedelta64[ns](1)
memory usage: 183.9+ KB


,date,end_time,result,time_control,num_moves,date_time,weekday,day,color,pre_elo,opp_pre_elo,diff,won_by
0,2018-04-10,16:09:44,0.5,600,130,2018-04-10 16:09:44,1,10,1,900,921,-21,stalemate
1,2018-04-10,16:25:54,0.5,600,169,2018-04-10 16:25:54,1,10,0,900,883,17,stalemate
2,2018-04-11,20:02:03,1.0,600,48,2018-04-11 20:02:03,2,11,0,929,952,-23,checkmate
3,2018-04-11,20:12:58,0.0,600,43,2018-04-11 20:12:58,2,11,0,938,911,27,checkmate
4,2018-04-12,20:10:08,0.0,600,98,2018-04-12 20:10:08,3,12,1,929,947,-18,time


In [143]:
def create_chess_df(ddf, mdf):
    cdf = pd.concat([ddf, mdf], axis=1)
    cdf = cdf.sort_values('date_time').reset_index().drop(
        columns=['index', 'date_time'])
    return cdf

cdf = create_chess_df(dfi, mfi)